# PDF Investor Summarizer

This notebook demonstrates how to use the [pdf-investor-summarizer](https://github.com/ashishki/pdf-investor-summarizer) project to automatically extract investment-relevant insights from company PDF reports using an LLM (OpenAI GPT).

**Features:**
- Loads and parses PDF files (local or public URL)
- Cleans and splits text for efficient LLM extraction
- Asks the LLM for a structured summary (growth, changes, triggers, material factors)
- Supports cost/token accounting and error handling

In [ ]:
# --- 1. Install project dependencies (always run first) ---

!git clone https://github.com/ashishki/pdf-investor-summarizer.git
%cd pdf-investor-summarizer
!pip install .

# (Optional) For OCR support in Colab:
# !apt-get install tesseract-ocr

In [3]:
# --- 2. Import modules ---
import os
import asyncio
from pathlib import Path
import requests
from src.pdf_investor_summarizer.report_analyzer import ReportAnalyzer



In [ ]:
# Set your OpenAI API key (required for the LLM)
os.environ["OPENAI_API_KEY"] = "sk-..."  # <-- Replace with your key!

In [ ]:
# Path to the default demo PDF (bundled with the repo)
PDF_SOURCE = "tests/assets/company_report.pdf"
print(f"Default demo PDF: {PDF_SOURCE}")

# Optional: If user uploads another file, use it instead
from google.colab import files
uploaded = files.upload()
if uploaded:
    PDF_SOURCE = list(uploaded.keys())[0]
    print(f"Using uploaded PDF: {PDF_SOURCE}")
else:
    print("No upload detected, using demo PDF.")


In [7]:
# --- 6. (Optional) Validate that PDF_SOURCE is a real PDF file ---
with open(PDF_SOURCE, "rb") as f:
    signature = f.read(4)
if not signature.startswith(b"%PDF"):
    raise ValueError("Selected file is not a valid PDF!")



In [8]:
# --- 7. Run the summarization pipeline ---
analyzer = ReportAnalyzer(chunk_size=2000, overlap=200)
result = await analyzer.analyze_async(PDF_SOURCE)

In [ ]:
# --- 8. Show the structured summary output ---
import json
print(json.dumps(result, indent=2, ensure_ascii=False))

In [10]:
#  (Optional) How much did this cost?

# Token/cost stats are already logged to console by the pipeline.
# If you want to analyze further, you can add custom print statements or save stats from result.

## Notes & Tips

- The pipeline supports both local files and public PDF URLs (except direct links from Google Drive/Dropbox/OneDrive — see README).
- If you get a PDF parsing error, make sure your input is a real PDF file, not a web page.
- For large files, costs and runtime may increase — try with a short report first.
- Adjust chunk size and overlap for your use-case: smaller values = cheaper but possibly less context.
- All token/cost stats and errors are printed in the output cells for transparency and easy debugging.
- For Google Drive, download your PDF to local disk and then upload via Colab.
- Poetry is used for development, but for Colab you only need pip install.
